In [1]:
import czifile
from skimage import io

In [31]:
path = 'RBC_tiny.czi'

with czifile.CziFile(path) as czi:
    
    
    # Extract raw metadata
    raw_metadata = czi.metadata()

    # If you want the metadata in dictionary form (easier to navigate), you can use:
    metadata_dict = czi.metadata(raw=False)
    channels_info = metadata_dict['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['Channels']
    
print(channels_info['Channel']['Id'])
print(channels_info['Channel']['ExcitationWavelength'])

Channel:0
488
